Abdullah Naveed i180654@nu.edu.pk

# Spell Correction for Roman Urdu

## Data Reading

In [1]:
from spacy.lang.en import English
enlp = English()
tokenizer = enlp.tokenizer

In [2]:
corpus = []

#Reading From Corpus file
file1 = open('data.txt', 'r')
corpusLines = file1.readlines()

# Save the text in List
for line in corpusLines:
    if(line != '\n'):
        corpus.append(line.strip())
    
        
# Testing
print("Length of Corpus Sentences: ", len(corpus))
print("Sample: ", corpus[532])

Length of Corpus Sentences:  4652818
Sample:  academy of urdu literature canada ne bhi unhe mein award diya


In [3]:
corpusWords = []

# Tokenizing corpus into a list of words
for line in corpus:
    if(line != '\n'):
        sentence = tokenizer(line)
        
        for word in sentence:
            if(word.text != "\t" and word.text != "\n"):
                corpusWords.append(word.text)
                
# Testing
print("Length of Corpus Words: ", len(corpusWords))
print("Sample: ", corpusWords[532])

Length of Corpus Words:  63308818
Sample:  gai


In [4]:
misspelledWords = []

#Reading From MisspelledWords file
file2 = open('misspellings.txt', 'r')
misspelledLines = file2.readlines()

# Tokenizing the words and saving in List
sentenceCounter = 0
for line in misspelledLines:
    if(line != '\n'):
        sentence = tokenizer(line)
        
        for index, word in enumerate(sentence):
            if(index == 0):
                misspelledWords.append([word.text])
                
            if(index > 1):
                if(word.text != "\t" and word.text != "\n"):
                    misspelledWords[sentenceCounter].append(word.text)
                
        sentenceCounter += 1
        

# Testing
print("Length of Misspelled Words: ", len(misspelledWords))
print("Sample:")
print(misspelledWords[0])
print(misspelledWords[1])

Length of Misspelled Words:  36101
Sample:
['Correct', 'Wrong']
['ka', 'kaz', 'cka', 'mka', 'kga', 'yka', 'kba']


# Language Models

### UNI-GRAM MODEL

In [5]:
unigrams = {}

# Formula of UNIGRAM: P(wi) = count(wi) / count(total number of words)

totalNumberOfWords = len(corpusWords)
unigramsCount = {}

# Getting Count for each word
for word in corpusWords:
    
    if word not in unigramsCount:
        unigramsCount[word] = 1
    else:
        unigramsCount[word] += 1

# Counting unigram probabilities
for word in unigramsCount:
    
    wordProbability = unigramsCount[word] / totalNumberOfWords
    unigrams[word] = wordProbability

# Testing
print("Length of UNIGRAM: ", len(unigrams))
print("Sample: ", unigrams["sai"])

Length of UNIGRAM:  35777
Sample:  2.503600683241314e-05


## Saving and Loading Models & Data (Optional)

In [6]:
#import json

# Saving models as json to avoid computation for later use.
#with open('JSON_DATA/UnigramModel.json', 'w') as f:
    #json.dump(unigrams,f)
    
# Other Data (to avoid reading and tokenization time)

#with open('JSON_DATA/corpus.json', 'w') as f:
    #json.dump(corpus,f)
    
#with open('JSON_DATA/corpusWords.json', 'w') as f:
    #json.dump(corpusWords,f)
    
#with open('JSON_DATA/misspelledWords.json', 'w') as f:
    #json.dump(misspelledWords,f)

In [7]:
#import json

# Loading models for use.
#with open('JSON_DATA/UnigramModel.json') as f:
    #unigrams = [tuple(x) for x in json.load(f)]
    
# Other Data

#with open('JSON_DATA/corpus.json') as f:
    #corpus = [(x) for x in json.load(f)]
    
#with open('JSON_DATA/corpusWords.json') as f:
    #corpusWords = [(x) for x in json.load(f)]
    
#with open('JSON_DATA/misspelledWords.json') as f:
    #misspelledWords = [(x) for x in json.load(f)]

### For Error Model

In [8]:
# Generating Bigrams

bigrams = []
unigramsForErrorModel = []
totalWords = set(corpusWords)

for index, row in enumerate(corpusWords):
    
    for letterNumber in range(0, len(row)-1):
        
        bigrams.append(row[letterNumber]+row[letterNumber+1])
        
        unigramsForErrorModel.append(row[letterNumber])
        if(letterNumber == len(row)-1):
            unigramsForErrorModel.append(row[letterNumber+1])

#Testing
print("Converted: ", corpusWords[0], "into:")
print(bigrams[0])
print(bigrams[1])

bigramsCount = {}
unigramsCountForErrorModel = {}

# Getting Count for each word
for word in bigrams:
    
    if word not in bigramsCount:
        bigramsCount[word] = 1
    else:
        bigramsCount[word] += 1
        
# Getting Count for each word
for word in unigramsForErrorModel:
    
    if word not in unigramsCountForErrorModel:
        unigramsCountForErrorModel[word] = 1
    else:
        unigramsCountForErrorModel[word] += 1

Converted:  sai into:
sa
ai


In [9]:
#Some Cleaning
bigrams.clear()
unigramsForErrorModel.clear()

## Pretty Tables Printings

In [10]:
import prettytable

# For Confusion Matrices
def print_confusionMatrix(confusionMatrix):
    
    tableCourses = prettytable.PrettyTable()
    tableCourses.field_names  = ['#','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    tableCourses._max_width = {'#':1,'a':1,'b':1,'c':1,'d':1,'e':1,'f':1,'g':1,'h':1,'i':1,'j':1,'k':1,'l':1,'m':1,'n':1,'o':1,'p':1,'q':1,'r':1,'s':1,'t':1,'u':1,'v':1,'w':1,'x':1,'y':1,'z':1}
    counter = 0
    for rowCount,row in enumerate(confusionMatrix):
        
        if(rowCount > 0):
            items = []
            spaces = []
            for itemCount, item in enumerate(row):
                items.append(item)
                spaces.append(" ")

            tableCourses.add_row(items)
            tableCourses.add_row(spaces)
            counter += 1
        
    print(tableCourses)
    

# For Selection Model
def print_SelectionModel(candidateWords, wrongCharacters, correctCharacters, candidateProbabilityFromErrorModel, candidateProbabilityFromLanguageModel, candidateFinalProbability):
    
    tableCourses = prettytable.PrettyTable()
    tableCourses.field_names  = ['Candidates','x|w','P(x|w)','P(w)','P(x|w)*P(w) *10^9']

    for rowCount,row in enumerate(candidateWords):
        items = []
        
        items.append(row)
        items.append(wrongCharacters[rowCount] + "|" + correctCharacters[rowCount])
        items.append(candidateProbabilityFromErrorModel[rowCount])
        items.append(candidateProbabilityFromLanguageModel[rowCount])
        items.append(candidateFinalProbability[rowCount])

        tableCourses.add_row(items)
        
    print(tableCourses)

## Generating Confusion Matrices

In [11]:
# Initializing Confusion Matrices
import copy
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

insertionMatrix = []
deletionMatrix = []
substitutionMatrix = []
transpositionMatrix = []

# Generating 27*27 Confusion Matrices
for index in range(0,26):
    if(index == 0):
        insertionMatrix.append(["#"] + copy.deepcopy(alphabets))
        deletionMatrix.append(["#"] + copy.deepcopy(alphabets))
        substitutionMatrix.append(["#"] + copy.deepcopy(alphabets))
        transpositionMatrix.append(["#"] + copy.deepcopy(alphabets))
        
    insertionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    deletionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    substitutionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    transpositionMatrix.append(copy.deepcopy(alphabets) + ["0"])
    
tempAlphabets = copy.deepcopy(alphabets)
for index in range(0, 26):
    insertionMatrix[index+1][0] = tempAlphabets[index]
    deletionMatrix[index+1][0] = tempAlphabets[index]
    substitutionMatrix[index+1][0] = tempAlphabets[index]
    transpositionMatrix[index+1][0] = tempAlphabets[index]

# Changing inside values to zeros for later update
for indexOfRow, row in enumerate(insertionMatrix):
    for indexOfCol, col in enumerate(row):
        
        if(indexOfRow == 0 and indexOfCol == 0):
            insertionMatrix[indexOfRow][indexOfCol] = "#"
            deletionMatrix[indexOfRow][indexOfCol] = "#"
            substitutionMatrix[indexOfRow][indexOfCol] = "#"
            transpositionMatrix[indexOfRow][indexOfCol] = "#"
        
        if(indexOfRow > 0 and indexOfCol > 0):
            insertionMatrix[indexOfRow][indexOfCol] = 0
            deletionMatrix[indexOfRow][indexOfCol] = 0
            substitutionMatrix[indexOfRow][indexOfCol] = 0
            transpositionMatrix[indexOfRow][indexOfCol] = 0
            
# Testing
for row in transpositionMatrix:
    for item in row:
        print(item, end= " ")
    print()
    
#print(alphabets)

# a b c d e f g h i j k l m n o p q r s t u v w x y z 
a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
b 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
c 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
d 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
e 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
f 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
g 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
h 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
i 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
j 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
k 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
l 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
m 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
n 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
o 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
p 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
q 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
r 0 0 0 0 

In [12]:
# Helpers

def change_item_string(string, new_character, position):
    string = string[:position] + new_character + string[position+1:]
    return string

def add_item_string(string, new_character, position):
    string = string[:position] + new_character + string[position:]
    return string

def remove_item_string(string, position):
    string = string[:position] + string[position+1:]
    return string

In [13]:
# Filling Confusion Matrices

def find_edit_type(correctWord, wrongWord):
    
    #To store type         // 0: insertion, 1: deletion, 2: substitution, 3: Transposition
    editType = [False]*4
    
    wi = ""
    xi = ""
    
    # If both words are of equal length
    if(len(correctWord) == len(wrongWord)):
        
        for index in range(0, len(correctWord)):
            
            # If an alphabet doesn't match
            if(correctWord[index] != wrongWord[index]):
                
                #Now its either substituted or transposed
                
                # For Substitution:
                tempWord = wrongWord
                
                tempWord = change_item_string(tempWord, correctWord[index], index)
                
                if(tempWord == correctWord):
                    editType[2] = True
                    wi = correctWord[index]
                    xi = wrongWord[index]
                    break
                    
                # For Transposition
                tempWord = wrongWord
                if(index < len(tempWord)-1):
                    tempAlphabet = tempWord[index]
                    tempWord = change_item_string(tempWord, tempWord[index+1], index)            
                    tempWord = change_item_string(tempWord, tempAlphabet, index+1)

                    if(tempWord == correctWord):
                        editType[3] = True
                        wi = wrongWord[index+1] + wrongWord[index]
                        xi = wrongWord[index] + wrongWord[index+1]
                        break
                    
    
    # If both words are of different length
    else:
        
        ## Checking for last alphabet
        
        # Check if alphabet inserted at the end
        if(wrongWord[:len(wrongWord)-1] == correctWord):
            editType[0] = True
            wi = wrongWord[len(wrongWord)-2:len(wrongWord)-1]
            xi = wrongWord[len(wrongWord)-1:len(wrongWord)]
            #print(correctWord, "misspelled as:", wrongWord, "by inserting:", xi, "after:", wi)
            return False, editType, wi, xi

        # Check if alphabet deleted at the end
        elif(correctWord[:len(correctWord)-1] == wrongWord):
            editType[1] = True
            wi = correctWord[len(correctWord)-2:len(correctWord)-1]
            xi = correctWord[len(correctWord)-1:len(correctWord)]
            #print(correctWord, "misspelled as:", wrongWord, "by deleting:", xi, "after:", wi)
            return False, editType, wi, xi
        
        ## Checking before last alphabet
        else:
            shorterWordLength = min(len(correctWord), len(wrongWord))
            
            for index in range(0, shorterWordLength):

                # If an alphabet doesn't match
                if(correctWord[index] != wrongWord[index]):

                    # For Deletion
                    tempWord = wrongWord
                    tempWord = add_item_string(tempWord, correctWord[index], index)

                    if(tempWord == correctWord):
                        editType[1] = True
                        xi = correctWord[index]
                        wi = wrongWord[index]
                        break
                        
                    # For Insertion
                    tempWord = wrongWord
                    tempWord = remove_item_string(tempWord, index)

                    if(tempWord == correctWord):
                        editType[0] = True
                        xi = wrongWord[index]
                        wi = correctWord[index]
                        break
    
    #Testing
    #if(editType[0]):
        #print(correctWord, "misspelled as:", wrongWord, "by inserting:", xi, "before:", wi)
        
    #elif(editType[1]):
        #print(correctWord, "misspelled as:", wrongWord, "by deleting:", xi, "before:", wi)
        
    #elif(editType[2]):
        #print(correctWord, "misspelled as:", wrongWord, "by substituting:", xi, "instead of:", wi)
        
    #elif(editType[3]):
        #print(correctWord, "misspelled as:", wrongWord, "by tranposition of:", xi, "with:", wi)
        
    return True, editType, wi, xi

In [14]:
# Testing

#find_edit_type("hello","htello")
#print()
#find_edit_type("hello","ello")
#print()
#find_edit_type("hello","herlo")
#print()
#find_edit_type("hello","helol")

In [15]:
# Updating tables for "misspellings.txt"

# Here: Wrong word on Y-AXIS & Correct word on X-AXIS
indexes = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,'t':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26}

for index, row in enumerate(misspelledWords):
    
    # First row is: "Correct, Wrong" so skipping it!
    if(index == 0):
        continue
        
    for wordNumber, word in enumerate(row):
        
        if(wordNumber > 0 and word != ","):
            
            # Check for error
            errorBeforeEnd, editType, wi, xi = find_edit_type(row[0], word)
            
            if(errorBeforeEnd):
                
                #Insertion of xi before wi
                if(editType[0]):
                    
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    insertionMatrix[X_AXIS][Y_AXIS] += 1
                    
                #Deletion of xi before wi
                elif(editType[1]):
                    
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    deletionMatrix[X_AXIS][Y_AXIS] += 1
                    
                #Substitution of xi instead of wi
                elif(editType[2]):
                    
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    substitutionMatrix[X_AXIS][Y_AXIS] += 1
                    
                #Transposition of xi with wi
                elif(editType[3]):
    
                    Y_AXIS = indexes[wi[0]]
                    X_AXIS = indexes[xi[0]]
                    transpositionMatrix[X_AXIS][Y_AXIS] += 1
                    
            else:
                #Insertion of xi after wi
                if(editType[0]):
                    
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    insertionMatrix[X_AXIS][Y_AXIS] += 1
                
                #Deletion of xi after wi
                elif(editType[1]):
                    
                    Y_AXIS = indexes[wi]
                    X_AXIS = indexes[xi]
                    deletionMatrix[X_AXIS][Y_AXIS] += 1
                    

In [16]:
#Testing a confusion Matrix
print_confusionMatrix(substitutionMatrix)

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| # | a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z |
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
| a | 0 | 7 | 6 | 1 | 1 | 2 | 4 | 1 | 1 | 3 | 8 | 1 | 9 | 2 | 1 | 5 | 1 | 1 | 1 | 1 | 9 | 1 | 2 | 1 | 7 | 2 |
|   |   | 8 | 6 | 1 | 8 | 8 | 8 | 8 | 9 | 2 | 1 | 3 | 2 | 0 | 2 | 9 | 5 | 9 | 4 | 5 | 9 | 7 | 9 |   | 3 | 6 |
|   |   |   |   | 9 | 4 |   |   | 9 | 0 |   |   | 8 |   | 1 | 9 |   |   | 3 | 5 | 9 |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
| b | 6 | 0 | 6 | 1 | 2 | 2 | 6 | 1 | 2 | 2 | 8 | 1 | 8 | 2 | 2 | 6 | 1 | 2 | 1 | 1 | 1 | 2 | 2 | 5 | 8 | 3 |
|   | 4 |   | 5 | 1 | 4 | 4 | 1 | 9 | 5 | 5 | 8 | 3 | 9 | 3 | 0 | 0 | 5 | 1 | 7 | 8 | 0 | 3 | 8 |   | 5 | 1 |
|   | 2 | 

## Error Model P(x|w)

In [17]:
def error_model(correctWord, wrongWord):
    
    errorBeforeEnd, editType, wi, xi = find_edit_type(correctWord, wrongWord)
       
    #Insertion of xi before wi
    if(editType[0]):
              
        # P(x|w) = ins(wi-1, xi) / count(wi-1)
        indexOfInsertedLetter = wrongWord.index(xi) #xi
        indexOfInsertedAfter = indexOfInsertedLetter - 1 #wi-1
        
        Y_AXIS = indexes[correctWord[indexOfInsertedAfter].lower()]
        X_AXIS = indexes[wrongWord[indexOfInsertedLetter].lower()]
 
        countWord = unigramsCountForErrorModel[correctWord[indexOfInsertedAfter]]

        if(countWord == 0):
            return (insertionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, "#"  
        else:  
            return (insertionMatrix[X_AXIS][Y_AXIS] / countWord), xi, "#"           
            
    #Deletion of xi before wi
    elif(editType[1]):
            
        # P(x|w) = del(wi-1, wi) / count(wi-1 wi)
        indexOfDeletedLetter = correctWord.index(xi) #wi
        indexOfDeletedAfter = indexOfDeletedLetter - 1 #wi-1
        
        Y_AXIS = indexes[correctWord[indexOfDeletedAfter].lower()]
        X_AXIS = indexes[correctWord[indexOfDeletedLetter].lower()]
 
        countWord = bigramsCount[correctWord[indexOfDeletedAfter] + correctWord[indexOfDeletedLetter]]

        if(countWord == 0):
            return (deletionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), "#", xi  
        else:  
            return (deletionMatrix[X_AXIS][Y_AXIS] / countWord), "#", xi  
                    
    #Substitution of xi instead of wi
    elif(editType[2]):
            
        # P(x|w) = sub(xi, wi) / count(wi)
        indexOfSubstitutedLetter = wrongWord.index(xi) #xi
        indexOfSubstitutedAfter = correctWord.index(wi) #wi
        
        Y_AXIS = indexes[correctWord[indexOfSubstitutedAfter].lower()]
        X_AXIS = indexes[wrongWord[indexOfSubstitutedLetter].lower()]
 
        countWord = unigramsCountForErrorModel[correctWord[indexOfSubstitutedAfter]]

        if(countWord == 0):
            return (substitutionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, wi  
        else:  
            return (substitutionMatrix[X_AXIS][Y_AXIS] / countWord), xi, wi                     
                
                    
    #Transposition of xi with wi
    elif(editType[3]):
            
        # P(x|w) = sub(wi, wi+1) / count(wi wi+1)
        indexOfTransposedLetter = correctWord.index(wi) #wi
        indexOfTransposedBefore = indexOfTransposedLetter + 1 #wi+1
        
        Y_AXIS = indexes[correctWord[indexOfTransposedBefore].lower()]
        X_AXIS = indexes[correctWord[indexOfTransposedLetter].lower()]
 
        countWord = bigramsCount[correctWord[indexOfTransposedLetter] + correctWord[indexOfTransposedBefore]]

        if(countWord == 0):
            return (transpositionMatrix[X_AXIS][Y_AXIS] / 0 + len(corpusWords)), xi, wi  
        else:  
            return (transpositionMatrix[X_AXIS][Y_AXIS] / countWord), xi, wi                
                  

In [18]:
print(error_model("martaba", "martwaba"))

(3.086100475037274e-05, 'w', '#')


## Generating Candidate Words

In [19]:
def edit_distance(correctWord, wrongWord):
    
    # If both words are of equal length
    if(len(correctWord) == len(wrongWord)):
        
        for index in range(0, len(correctWord)):
            
            # If an alphabet doesn't match
            if(correctWord[index] != wrongWord[index]):
                
                #Now its either substituted or transposed
                
                # For Substitution:
                tempWord = wrongWord
                
                tempWord = change_item_string(tempWord, correctWord[index], index)
                
                if(tempWord == correctWord):
                    return 1
                    
                # For Transposition
                tempWord = wrongWord
                if(index < len(tempWord)-1):
                    tempAlphabet = tempWord[index]
                    tempWord = change_item_string(tempWord, tempWord[index+1], index)            
                    tempWord = change_item_string(tempWord, tempAlphabet, index+1)

                    if(tempWord == correctWord):
                        return 1
                    
    
    # If both words are of different length
    else:
        
        ## Checking for last alphabet
        
        # Check if alphabet inserted at the end
        if(wrongWord[:len(wrongWord)-1] == correctWord):
            return 1

        # Check if alphabet deleted at the end
        elif(correctWord[:len(correctWord)-1] == wrongWord):
            return 1
        
        ## Checking before last alphabet
        else:
            shorterWordLength = min(len(correctWord), len(wrongWord))
            
            for index in range(0, shorterWordLength):

                # If an alphabet doesn't match
                if(correctWord[index] != wrongWord[index]):

                    # For Deletion
                    tempWord = wrongWord
                    tempWord = add_item_string(tempWord, correctWord[index], index)

                    if(tempWord == correctWord):
                        return 1
                        
                    # For Insertion
                    tempWord = wrongWord
                    tempWord = remove_item_string(tempWord, index)

                    if(tempWord == correctWord):
                        return 1
    
    return 0

In [20]:
def FindCandidateWords(wrongWord):
    
    candidateWords = []
    
    # Checking for 1 edit distance words in vocabulary
    for word in totalWords:
        
        editDistances = edit_distance(word, wrongWord)
        if(editDistances == 1):
            candidateWords.append(word)
            
    return candidateWords

## Selection Model

In [21]:
def all_possible_correct_word(wrongWord):
    
    candidateWords = FindCandidateWords(wrongWord)
    
    candidateProbabilityFromLanguageModel = []
    candidateProbabilityFromErrorModel = []
    candidateFinalProbability = []
    
    wrongCharacters = []
    correctCharacters = []
    
    for correctWord in candidateWords:
        probability, wrongChar, correctChar = error_model(correctWord, wrongWord)
            
        candidateProbabilityFromErrorModel.append(probability)
        wrongCharacters.append(wrongChar)
        correctCharacters.append(correctChar)
        candidateProbabilityFromLanguageModel.append(unigrams[correctWord])
        
        candidateFinalProbability.append((probability * unigrams[correctWord]) * pow(10, 9))
        
    
    print_SelectionModel(candidateWords, wrongCharacters, correctCharacters, candidateProbabilityFromErrorModel, candidateProbabilityFromLanguageModel, candidateFinalProbability)
    
    return candidateWords, candidateFinalProbability

In [22]:
def best_possible_word(wrongWord):
    
    candidateWords, candidateFinalProbability = all_possible_correct_word(wrongWord)
    print("\n\n\n")
    
    if(candidateFinalProbability):
        max_value = max(candidateFinalProbability)
    else:
        return "None"
    candidateWord = candidateWords[candidateFinalProbability.index(max_value)]
    
    return candidateWord

## RESULTS

In [23]:
wrongWord = input("Enter Wrong Word:")

bestWord = best_possible_word(wrongWord)
    
print("Best Correction for:", wrongWord, "ïs:", bestWord)

Enter Wrong Word:hax
+------------+-----+------------------------+------------------------+-----------------------+
| Candidates | x|w |         P(x|w)         |          P(w)          |   P(x|w)*P(w) *10^9   |
+------------+-----+------------------------+------------------------+-----------------------+
|    hnx     | a|n | 2.138072237483023e-05  | 1.2636470325508209e-06 |  0.027017686382747156 |
|    hab     | x|b | 1.756632948422524e-05  | 6.950058679029515e-07  |  0.01220870206905317  |
|    haz     | x|z | 1.794181581268296e-05  | 4.2648087348590207e-07 |  0.007651841279716198 |
|    hanx    | #|n | 5.610979564812425e-06  |  6.63414692089181e-07  |  0.003722406280308722 |
|    had     | x|d | 3.0866644327210414e-05 | 0.00032210362859720425 |   9.942258140413784   |
|    oax     | h|o | 1.750414464967514e-05  | 3.159117581377053e-08  |  0.000552976511097558 |
|    haq     | x|q | 1.462189010674792e-05  | 9.737979944594764e-05  |   1.4238767261157985  |
|    haj     | x|j | 1.533957

In [24]:
wrongWord = input("Enter Wrong Word:")

bestWord = best_possible_word(wrongWord)
    
print("Best Correction for:", wrongWord, "ïs:", bestWord)

Enter Wrong Word:humar
+------------+-------+------------------------+------------------------+----------------------+
| Candidates |  x|w  |         P(x|w)         |          P(w)          |  P(x|w)*P(w) *10^9   |
+------------+-------+------------------------+------------------------+----------------------+
|    umar    |  h|#  | 2.8826290297408496e-05 | 3.768827274582824e-05  |  1.0864130909791536  |
|   humay    |  r|y  | 1.6610810049407192e-05 | 5.844367525547547e-07  | 0.009707967882579424 |
|   kumar    |  h|k  | 7.596017651018136e-06  | 3.854123449280004e-06  | 0.02927598974993381  |
|   shumar   |  #|s  |          0.0           | 1.1688735051095094e-06 |         0.0          |
|   homar    |  u|o  | 2.0249892830016335e-05 | 1.5795587906885263e-06 | 0.03198589623015286  |
|   hamar    |  u|a  | 1.3531663031409298e-05 | 2.3693381860327894e-07 | 0.003206108594084626 |
|   humor    |  a|o  | 1.475839646933394e-05  | 1.453194087433444e-06  | 0.021446814489234696 |
|   humari   |  #

In [25]:
wrongWord = input("Enter Wrong Word:")

bestWord = best_possible_word(wrongWord)
    
print("Best Correction for:", wrongWord, "ïs:", bestWord)

Enter Wrong Word:kaya
+------------+-------+------------------------+------------------------+-----------------------+
| Candidates |  x|w  |         P(x|w)         |          P(w)          |   P(x|w)*P(w) *10^9   |
+------------+-------+------------------------+------------------------+-----------------------+
|    aya     |  k|#  |  1.58596910798238e-05  | 5.514239738293645e-05  |   0.8745413878942565  |
|   kayam    |  #|m  | 1.730050490442282e-05  | 4.422764613927873e-07  |  0.007651606089436687 |
|    kama    |  y|m  | 1.4797555981844743e-05 | 3.190708757190823e-06  |  0.04721469145629347  |
|    kaza    |  y|z  | 1.7381134068536616e-05 | 1.9112661367331165e-06 |  0.03321997296321233  |
|    kaga    |  y|g  | 2.1825418844584523e-05 |  7.89779395344263e-08  | 0.0017237266098211249 |
|    haya    |  k|h  | 1.0978673204314823e-05 | 8.150523359952796e-06  |  0.08948193241305578  |
|    gaya    |  k|g  | 2.3305108257776693e-05 | 0.0028844638988521315  |   67.22274342839756   |
|    aay

In [26]:
wrongWord = input("Enter Wrong Word:")

bestWord = best_possible_word(wrongWord)
    
print("Best Correction for:", wrongWord, "ïs:", bestWord)

Enter Wrong Word:asdfasdfas
+------------+-----+--------+------+-------------------+
| Candidates | x|w | P(x|w) | P(w) | P(x|w)*P(w) *10^9 |
+------------+-----+--------+------+-------------------+
+------------+-----+--------+------+-------------------+




Best Correction for: asdfasdfas ïs: None


In [27]:
wrongWord = input("Enter Wrong Word:")

bestWord = best_possible_word(wrongWord)
    
print("Best Correction for:", wrongWord, "ïs:", bestWord)

Enter Wrong Word:hampe
+------------+-----+------------------------+------------------------+-----------------------+
| Candidates | x|w |         P(x|w)         |          P(w)          |   P(x|w)*P(w) *10^9   |
+------------+-----+------------------------+------------------------+-----------------------+
|   sampe    | h|s | 1.751839401779987e-05  | 2.3693381860327894e-07 |  0.004150699990434161 |
|   hamle    | p|l | 2.2871970639767755e-05 | 6.160279283685252e-07  |  0.014089772690921862 |
|   hamre    | p|r | 2.8150674118562983e-05 | 6.318235162754104e-07  |  0.017786257907113655 |
|   hamne    | p|n | 2.3401785683893784e-05 | 1.6111499665022968e-06 |   0.0377037862206994  |
|   humpe    | a|u | 1.5474216750636436e-05 | 3.317073460445905e-07  |  0.00513291137047236  |
|    hame    | p|# | 1.8235372018030897e-05 | 1.4610918813868867e-05 |   0.2664355400961455  |
|    ampe    | h|# | 2.9535187910042257e-05 | 1.263647032550821e-07  | 0.0037322052558355786 |
|   hamse    | p|s | 1.9530